In [1]:
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
import matplotlib.pyplot as plt
import keras



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
data_folder_path = '/content/drive/My Drive/Data'

In [ ]:
# build analyzers (bag-of-words)
BOW_vectorizer = CountVectorizer()

In [ ]:
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize)

In [ ]:
#Import JSON File

import json

# Opening JSON file
tweets = []
for line in open('/content/drive/My Drive/Data/tweets_DM.json', 'r'):
    tweets.append(json.loads(line))

In [ ]:
df = pd.DataFrame(tweets)

In [ ]:
df

In [ ]:
#Remove -type column
df = df.drop('_type', axis=1)

In [ ]:
df = df.drop('_index', axis=1)

In [ ]:
df

In [ ]:
df_flatten = pd.DataFrame(df._source.values.tolist())
df_new = pd.DataFrame(df_flatten.tweet.values.tolist())

In [ ]:
df_new

In [ ]:
data_identification = pd.read_csv('/content/drive/My Drive/Data/data_identification.csv')
emotion = pd.read_csv('/content/drive/MyDrive/Data/emotion.csv')

In [ ]:
df_merge = pd.merge(df_new, data_identification, on=['tweet_id'])

In [ ]:
train_df = df_merge[df_merge['identification'] == 'train']
test_df = df_merge[df_merge['identification'] == 'test']

In [ ]:
train_df = pd.merge(train_df, emotion, on=['tweet_id'])

In [ ]:
train_df.to_pickle("train_df.pkl")
test_df.to_pickle("test_df.pkl")

In [ ]:
train_df = pd.read_pickle("train_df.pkl")
test_df = pd.read_pickle("test_df.pkl")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
analy_count = CountVectorizer(tokenizer= nltk.word_tokenize)
analy_count.fit(train_df['text'])

X_train = analy_count.transform(train_df['text'])
y_train = train_df['emotion']

X_test = analy_count.transform(test_df['text'])

In [ ]:
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
import keras

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)

In [ ]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Dropout
from keras.layers import ReLU, Softmax

model = Sequential()
model.add(Dense(64, input_shape=(input_shape,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(units = output_shape))
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

In [ ]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 7
batch_size = 128

# training!
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose = 1,
                    callbacks=[csv_logger],)
print('training finish')

In [ ]:
pred_result = model.predict(X_test, batch_size=128)
pred_result[:5]